# Annexe

Dans l'OpenData d'IDFM, en plus des validations sur le réseau ferré, il y a aussi des données de validation sur le réseau de surface, c'est-à-dire les lignes de bus et de tramway. Mais ces données de validations sont différentes de celles du réseau ferré. Pour le réseau ferré nous avions des données sur des stations (La Défense, Châtelet...), alors que pour le réseau de surface nous avons des données sur des lignes (le bus 30, le tramway T2...) car les valideurs sont situés directement à bord des bus ou des tramways. Ces données ne rentrent donc pas dans notre sujet, mais peuvent tout de même montrer des résultats intéressants. On se propose de les étudier dans une annexe.

### Importations

In [22]:
import pandas as pd
import numpy as np

## I. Validations sur le réseau de surface (RDS)

In [23]:
df_rds = pd.read_csv("data/validations/validations_rds.csv",sep = ";")
df_rds = df_rds.rename(columns={"jour":"JOUR","libelle_ligne":"LIBELLE_LIGNE","id_groupofligne":"ID_GROUPOFLIGNE","nb_vald":"NB_VALD"})

vald_rds = (
    df_rds.groupby(['ID_GROUPOFLIGNE', 'LIBELLE_LIGNE'], as_index=False)['NB_VALD']
    .sum()
    .sort_values(by='NB_VALD', ascending=False)
)

# Chaque ligne a un identifiant unique : ID_GROUPOFLIGNE

vald_rds['NB_VALD_MOY_JOUR'] = (vald_rds['NB_VALD']/91).round(0) #pour arrondir : round(0)
vald_rds = vald_rds[vald_rds["ID_GROUPOFLIGNE"] != "A00000"] #on enlève les lignes non-définies
vald_rds.head(10)

,ID_GROUPOFLIGNE,LIBELLE_LIGNE,NB_VALD,NB_VALD_MOY_JOUR
981,A01193,T3a,10700314,117586.0
1228,A01757,T3b,10662056,117165.0
980,A01192,T2,8621139,94738.0
1414,A02371,T9 - Paris Porte de Choisy - Orly Gaston Viens,5622812,61789.0
736,A00924,TVM,4912649,53985.0
1268,A02149,T6,4029208,44277.0
763,A00952,62,2776453,30510.0
1269,A02150,T8,2488586,27347.0
1231,A01760,T5,2460480,27038.0
761,A00950,60,1821021,20011.0


Note : toutes les lignes qui ont un nom commençant par un T sont des tramways, sauf le TVM qui est un bus.

Les lignes les plus fréquentées du réseau de surface sont surtout des lignes de tramways, c'est plutôt logique car un tramway peut contenir bien plus de personnes qu'un bus.
La ligne la plus fréquentée du réseau de surface est le T3a, suivie de près par le T3b. Ces deux lignes réunies ont un trajet faisant le tour de Paris. Ce sont les seules lignes de tramways ayant un trajet intégralement intra-muros.

La ligne de bus la plus fréquentée d'Ile de France est le TVM (TVM est pour Trans Val de Marne). Comme son nom l'indique, elle traverse le département du Val de Marne d'Est en Ouest. C'est une des rares lignes d'Ile de France pouvant être qualifiée de BHNS (Bus à Haut Niveau de Service), car elle bénéficie d'un site propre sur la quasi-intégralité de son trajet. C'est-à-dire que les bus roulent sur une voie qui leur exclusivement réservée, et ont la priorité aux intersections. Cela permet de gagner énormément de temps sur le trajet.

Remarque : En plus d'être le bus le plus fréquenté d'Ile de France comme on le constate ici, le TVM est le bus le plus fréquenté d'Europe !

## II. Fréquence des lignes

In [24]:
# Calcul de l'offre. Dans le GTFS il y a réseau ferré et réseau de surface.
agency = pd.read_csv("data/gtfs/agency.txt") #associe agency_id avec réseau de transport. Go trouver Paris-Saclay
routes = pd.read_csv("data/gtfs/routes.txt") #associe route_id avec une ligne de transport
trips = pd.read_csv("data/gtfs/trips.txt") #lien avec routes par route_id / calendar par service_id / stop_time par trip_id
calendar = pd.read_csv("data/gtfs/calendar.txt") #quel jour il y a tel service, identifié par service_id
calendar_dates = pd.read_csv("data/gtfs/calendar_dates.txt") #services à ajouter/retirer car exceptions
stops = pd.read_csv("data/gtfs/stops.txt") #associe stop_id avec un arrêt physique
stop_times = pd.read_parquet("data/gtfs/stop_times.parquet")

/tmp/ipykernel_207462/2975124623.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  trips = pd.read_csv("data/gtfs/trips.txt") #lien avec routes par route_id / calendar par service_id / stop_time par trip_id


In [25]:
services_jour = (calendar.groupby(
    ["start_date"],as_index=False)["service_id"].nunique()
) #nombre de services qui commencent tel jour

#On prend le lundi 8 décembre, il a l'air bien au milieu de tout

date = pd.Timestamp("2025-12-08")
dint = int(date.strftime("%Y%m%d")) #conversion au même format que les dates des données GTFS

#Services actifs le 08/12
actifs = calendar[
    (calendar["start_date"] <= dint) &
    (calendar["end_date"]   >= dint) &
    (calendar["monday"] == 1)
]["service_id"]

#Exceptions du 08/12
exceptions = calendar_dates[calendar_dates["date"] == dint]
actifs = pd.Index(actifs).unique()

enlever = pd.Index(exceptions.loc[exceptions["exception_type"] == 2, "service_id"]).unique()
ajouter = pd.Index(exceptions.loc[exceptions["exception_type"] == 1, "service_id"]).unique()
#loc : sert à sélectionner certaines lignes et certaines colonnes, avec potentiellement des booléens

actifs = actifs.difference(enlever)
actifs = actifs.union(ajouter)
d_actifs = {}
for service_id in actifs :
    d_actifs[service_id] = 1

trips_0812 = trips.loc[trips["service_id"].isin(actifs)]
#isin : vérifie pour chaque élément du DataFrame s'il est dans actifs

offre = trips_0812.merge(routes,on="route_id",how="left")
offre = offre[["route_id","route_long_name","trip_id","direction_id"]]

# Heure (ou minute) de départ du trip au premier arrêt
st = stop_times.loc[stop_times["trip_id"].isin(offre["trip_id"])].copy()
st = st.sort_values(["trip_id", "stop_sequence"])
first_dep = st.groupby("trip_id", as_index=False).first()[["trip_id", "departure_time"]]

# Conversion de HH:MM:SS en minutes depuis 00:00 (gère par ex "25:10:00" si nécessaire)
def to_min(t):
    #marche même si t dépasse 24:00:00
    h, m, s = map(int, t.split(":"))
    return h*60 + m + s/60

first_dep["dep_min"] = first_dep["departure_time"].map(to_min)

dep = first_dep.merge(offre[["trip_id","route_id","direction_id"]], on="trip_id", how="left")

# Pointe du matin 7h–10h
peak = dep[(dep["dep_min"] >= 7*60) & (dep["dep_min"] < 10*60)]

freq = (peak.groupby(["route_id","direction_id"], as_index=False)
             .size()
             .rename(columns={"size": "voyages_3h"}))
freq["voyages_h"] = freq["voyages_3h"] / 3.0
freq["headway_min"] = 60.0 / freq["voyages_h"]

freq = freq.merge(
    routes[["route_id","route_type","route_long_name"]],
    on="route_id", how="left"
)
freq = freq.sort_values(by="voyages_h",ascending=False)
#chaque ligne apparait 2 fois dans la liste, une fois pour chaque sens

freq = freq.drop_duplicates(subset="route_long_name",keep="first") 
freq.head(10)

,route_id,direction_id,voyages_3h,voyages_h,headway_min,route_type,route_long_name
2065,IDFM:C01742,1,138,46.000000,1.304348,2,A
2063,IDFM:C01740,1,114,38.000000,1.578947,2,L
2054,IDFM:C01728,1,107,35.666667,1.682243,2,D
1664,IDFM:C01371,1,105,35.000000,1.714286,1,1
1687,IDFM:C01383,0,103,34.333333,1.747573,1,13
2066,IDFM:C01743,1,102,34.000000,1.764706,2,B
2059,IDFM:C01737,1,102,34.000000,1.764706,2,H
2062,IDFM:C01739,1,101,33.666667,1.782178,2,J
2053,IDFM:C01727,1,100,33.333333,1.800000,2,C
1689,IDFM:C01384,0,100,33.333333,1.800000,1,14


Les lignes les plus fréquentes en heure de pointe du matin en semaine ne sont que des lignes ferrées. On peut l'expliquer que beaucoup de lignes ferrées d'Ile de France transportent énormément de voyageurs, ce qui nécessite une grande fréquence. Fréquence rendue possible par le fait d'avoir des voies dédiées à une unique ligne. La ligne la plus fréquente est le RER A, avec en moyenne un train toutes les 1.3 minutes sur le tronçon central.

Restreignons maintenant au réseau de surface :

In [26]:
freq_rds = freq[(freq["route_type"] == 0) | (freq["route_type"] == 3)]
freq_rds.head(15)

,route_id,direction_id,voyages_3h,voyages_h,headway_min,route_type,route_long_name
1139,IDFM:C01071,0,62,20.666667,2.903226,3,TVM
1272,IDFM:C01142,1,54,18.000000,3.333333,3,113
1701,IDFM:C01390,0,52,17.333333,3.461538,0,T2
2160,IDFM:C01843,0,50,16.666667,3.600000,0,T4
1703,IDFM:C01391,0,48,16.000000,3.750000,0,T3a
1238,IDFM:C01123,1,47,15.666667,3.829787,3,92
1993,IDFM:C01679,0,47,15.666667,3.829787,0,T3b
2110,IDFM:C01794,1,47,15.666667,3.829787,0,T6
1510,IDFM:C01266,1,46,15.333333,3.913043,3,291
1252,IDFM:C01132,1,45,15.000000,4.000000,3,103


Nous remarquons la présence de pas mal de lignes de tramway. Dans la même logique que les lignes ferrées, ces lignes transportent énormément de personnes donc doivent être très fréquentes, et bénéficient d'une voie dédiée. Cependant, la fréquence maximale d'une ligne de bus est plus élevée qu'une ligne de tramway, car le fait que le tramway soit contraint à rester sur des voies rend l'exploitation plus difficile. Il est par exemple impossible d'avoir deux tramways qui se suivent l'un collé à l'autre, alors que pour des bus cela ne pose pas de problème. 
C'est pour cette raison que la ligne la plus fréquente du réseau de surface est le TVM, qui combine à la fois le fait d'être un bus, et le fait d'avoir un site propre.

Autre remarque que nous pouvons faire sur ce classement : nous remarquons la présence de 5 lignes de bus exploitées avec des bus standards (113, 92, 291, 103, 129) et de 4 avec des bus articulés (TVM, Caméléon T1, 4606, 258). 
En effet, comme les bus standards peuvent transporter moins de personnes que les bus articulés, ils nécessitent une fréquence plus élevée sur les lignes très fréquentées.
(A notre connaissance il n'existe pas de base de données qui répertorie le matériel roulant de chaque ligne de bus, ces informations viennent de la page Wikipédia des lignes de bus RATP.)

Enfin, en tant qu'étudiants du Plateau de Saclay, nous constatons que la ligne 4606, que (presque) tout étudiant de l'ENSAE a déjà prise au moins une fois, est la 5ème ligne de bus la plus fréquente d'Ile de France en heure de pointe avec un bus tous les 4 minutes en moyenne. Cette ligne relie les campus universitaires et les logements du plateau à la gare de Massy-Palaiseau, point de passage quasiment indispensable pour se rendre à Paris depuis le plateau.
On peut rajouter que le tracé du 4606 va être entièrement repris par la ligne 18 du Grand Paris Express, qui devrait être mise en service fin 2026, avec moins de points d'arrêts. Le 4606 risque donc de voir son offre baisser et de ne plus être dans ce classement d'ici 1 an.

En effet, rappelons que dans ce classement nous regardons la fréquence de ces lignes en heure de pointe. Ainsi, la ligne 4606 n'est pas impactée par sa fréquence d'un bus tous les 15 minutes en heure creuse.

Etudions maintenant le réseau de bus de Paris-Saclay, car c'est le secteur de l'ENSAE, donc d'une certaine manière cela nous concerne directement !

In [34]:
routes = routes[["agency_id","route_id"]]
freq_routes = freq.merge(routes,on="route_id",how="left")
freq_saclay = freq_routes[freq_routes["agency_id"] == "IDFM:1060"] #restriction à Paris-Saclay
freq_saclay = freq_saclay[["route_id","route_long_name","voyages_h","headway_min"]]
freq_saclay

,route_id,route_long_name,voyages_h,headway_min
36,IDFM:C01567,4606,15.000000,4.000000
71,IDFM:C01561,4609,11.000000,5.454545
81,IDFM:C01697,4602,10.000000,6.000000
213,IDFM:C00495,4611,6.666667,9.000000
232,IDFM:C00068,4603,6.000000,10.000000
255,IDFM:C00687,4644,6.000000,10.000000
269,IDFM:C00072,4607,5.666667,10.588235
316,IDFM:C01698,4621,5.333333,11.250000
395,IDFM:C01699,4622,4.666667,12.857143
405,IDFM:C00069,4604,4.666667,12.857143


Nous remarquons trois lignes très fréquentes : le 4606 qu'on a déjà vu avant, ainsi que le 4609 et le 4602. 

Le 4609 complète la desserte du plateau de Saclay du 4606, en reliant le campus d'HEC et le secteur de l'université Paris-Saclay à la gare du Guichet, et à Orsay et aux Ulis.

Enfin, la ligne 4602 emprunte l'autoroute A10 depuis la gare de Massy-Palaiseau et dessert la zone industrielle de Courtaboeuf, ainsi que les villes d'Orsay et des Ulis.

Certaines lignes ont des fréquences très basses (par exemple, 1 bus tous les 3h en moyenne) car ce sont des lignes scolaires. Elles sont dédiées à une demande très ciblée.

## III. Lien avec les POI et limites

Nous nous demandons maintenant : est-ce que les lignes de bus et de tramway qui desservent beaucoup de POI sont les plus fréquentées ?

Nous avons pour cela le fichier shapes.csv qui, pour chaque ligne de transport d'Ile de France associe son tracé.

In [ ]:
import json
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, MultiLineString

def shape_geojson_to_geom(x):
    """
    Convertit une valeur de la colonne 'shape' (string JSON ou dict)
    au format GeoJSON (LineString / MultiLineString) en géométrie shapely.

    Attendu dans ton cas :
      {"coordinates": [[[lon,lat],...], [[lon,lat],...]], "type": "MultiLineString"}
    """

    if x is None or (isinstance(x, float) and np.isnan(x)):
        return None

    # Si c'est une string, la parser en dict
    if isinstance(x, str):
        try:
            x = json.loads(x)
        except Exception:
            return None

    if not isinstance(x, dict):
        return None

    geom_type = x.get("type")
    coords = x.get("coordinates")

    if coords is None:
        return None

    # helper : garder seulement lon/lat, même si [lon,lat,alt,...]
    def to_lonlat(pt):
        if isinstance(pt, (list, tuple)) and len(pt) >= 2:
            return (float(pt[0]), float(pt[1]))
        return None

    if geom_type == "LineString":
        pts = [to_lonlat(pt) for pt in coords]
        pts = [p for p in pts if p is not None]
        return LineString(pts) if len(pts) >= 2 else None

    if geom_type == "MultiLineString":
        lines = []
        for part in coords:  # part = [[lon,lat],...]
            pts = [to_lonlat(pt) for pt in part]
            pts = [p for p in pts if p is not None]
            if len(pts) >= 2:
                lines.append(LineString(pts))
        if not lines:
            return None
        return MultiLineString(lines) if len(lines) > 1 else lines[0]

    # Si jamais d'autres types apparaissent, on ignore
    return None

def poi_par_ligne(
    poi_path,
    rayon=200,
):
    """
    Calcule le nombre de POI à moins de "rayon" mètres du tracé (shape) de chaque ligne bus/tram.

    Requiert dans le fichier shapes :
      - route_id (et idéalement route_long_name)
      - route_type (ex: "Bus", "Tramway")
      - shape (GeoJSON-like avec coordinates lon/lat)
    Requiert dans les POI :
      - geometry (points), dans un parquet ou GeoDataFrame lisible par geopandas
    """

    # --- lire shapes ---
    df_shapes = pd.read_csv("data/ref/shapes.csv",sep=";")

    # --- filtrer bus/tram ---
    rt = df_shapes["route_type"].astype(str)
    is_bus = rt.isin(("Bus",))
    is_tram = rt.isin(("Tram",))
    df_shapes = df_shapes[is_bus | is_tram].copy()

    # --- géométries ---
    df_shapes["geometry"] = df_shapes["shape"].apply(shape_geojson_to_geom)
    df_shapes = df_shapes.dropna(subset=["geometry"])

    gdf_shapes = gpd.GeoDataFrame(df_shapes, geometry="geometry", crs="EPSG:4326")

    # --- reprojection métrique + buffer ---
    gdf_shapes_2154 = gdf_shapes.to_crs("EPSG:2154")
    gdf_shapes_2154["geometry"] = gdf_shapes_2154.geometry.buffer(rayon)

    # --- lire POI ---
    poi = gpd.read_parquet(poi_path)

    # assurer CRS métrique
    if poi.crs is None or poi.crs.to_string() != "EPSG:2154":
        poi = poi.set_crs("EPSG:4326", allow_override=True).to_crs("EPSG:2154")

    # identifiant POI (si tu n'en as pas)
    poi = poi.copy()
    poi["poi_id"] = poi.index

    # --- jointure spatiale : POI dans buffer ---
    hits = gpd.sjoin(
        poi[["poi_id", "geometry"]],
        gdf_shapes_2154[["route_id", "route_long_name","geometry"]],
        how="inner",
        predicate="within"
    )

    # --- compter les POI uniques par ligne ---
    out = (
        hits.groupby(["route_id", "route_long_name"])["poi_id"]
        .nunique()
        .reset_index()
        .rename(columns={"poi_id": f"nb_poi_{rayon}m"})
    )
    out = out.sort_values(by=f"nb_poi_{rayon}m",ascending=False)

    return out


In [21]:
logement_lignes = poi_par_ligne("data/poi/poi_logement.parquet")
logement_lignes.head(10)

,route_id,route_long_name,nb_poi_200m
1338,IDFM:C01841,P,9254
1638,IDFM:C02413,95-50 (future 1143),5959
1035,IDFM:C01418,N14,5954
1037,IDFM:C01420,N12,5876
1344,IDFM:C01847,L,5328
1036,IDFM:C01419,N13,5308
1342,IDFM:C01845,N,5287
1345,IDFM:C01848,R,5005
1343,IDFM:C01846,J,4882
1033,IDFM:C01416,N16,4667


En regardant les logements, on se rend compte qu'on va devoir nettoyer les données : dans le classement, nous voyons des lignes P, L... Ces lignes correspondent à des remplacements des transiliens de la même lettre par des bus, par exemple en cas de travaux. Ces lignes ne circulent qu'occasionnellement, on va donc les exclure.

Nous allons aussi exclure les lignes ayant un nom avec un N puis un nombre, car ce sont des noctiliens : des bus circulant uniquement la nuit.

Pour cela, nous allons utiliser le fichier ref_lignes.csv qui liste toutes les lignes de transport d'Ile de France avec toutes leurs caractéristiques.

In [47]:
ref = pd.read_csv("data/ref/ref_lignes.csv",sep=";")
ref = ref[["id_line","id_groupoflines","name_line","type"]]
ref = ref.rename(columns={"id_groupoflines":"ID_GROUPOFLIGNE"})
ref["route_id"] = "IDFM:" + ref["id_line"]
ref = ref[ref["type"].isna()] # Les bus de remplacements et autres lignes spéciales ont un "type" différent de NaN

filtre_special = logement_lignes.merge(ref,on="route_id",how="right")

filtre_nocti = filtre_special[~filtre_special["route_long_name"].str.contains("N", na=False)] 
# En réalité on enlève un peu plus que les noctiliens avec la condition "ne contient pas de N dans son nom", mais c'est négligeable

df_final = vald_rds.merge(filtre_nocti,on="ID_GROUPOFLIGNE",how="inner")

df_final = df_final[["route_id","route_long_name","nb_poi_200m","NB_VALD"]]

df_final = df_final.sort_values(by="nb_poi_200m",ascending=False)
df_final.head(10)

,route_id,route_long_name,nb_poi_200m,NB_VALD
1116,IDFM:C02413,95-50 (future 1143),5959.0,8109
176,IDFM:C01084,39,4600.0,437066
129,IDFM:C01072,20,4439.0,577234
146,IDFM:C01093,56,3776.0,512622
1308,IDFM:C02555,6559,3747.0,4042
1387,IDFM:C02206,95-48 (future 1117),3693.0,3112
529,IDFM:C00774,1207,3674.0,95567
14,IDFM:C01083,38,3631.0,1605096
44,IDFM:C01073,21,3615.0,1012398
170,IDFM:C01079,29,3503.0,452478


On retrouve deux types de lignes dans celles qui desservent le plus de logements : des lignes de Paris intra-muros (toutes celles à deux chiffres) et des lignes de banlieue très longues.

FAIRE UNE REGRESSION AVEC TOUS LES POI

Les lignes de bus sont très nombreuses et très variées, et d'autres variables que l'on ne prend pas en compte avec nos données semblent pouvoir expliquer leur fréquentation.

Par exemple, en reprenant le classement des lignes du réseau de surface les plus fréquentées, dans le top10 apparaissent deux lignes de bus parisiennes : le 62 et le 60. Ces deux lignes desservent des zones géographiquement différentes (le Sud pour le 62 et le Nord-Est pour le 60) mais quand on regarde leur trajet, ces deux lignes ont un point commun : elles desservent des zones mal desservies par le métro et ont un trajet en rocade. Alors que le réseau de métro de Paris est principalement radial (un problème que va tenter de résoudre la ligne 15 du Grand Paris Express), le 62 et le 60 permettent des trajets qui sont contraignants à faire en métro. Au vu de la très forte densité du réseau de métro parisien, ces cas sont rares.
Ces deux lignes ne desservent pas de grand pôle, elles répondent surtout à une demande résidentielle qui n'a pas de meilleure alternative.

Ainsi, un critère que l'on pourrait prendre en compte pour expliquer la fréquentation des lignes de bus serait le manque d'alternatives ferrées. Cependant, cela semble difficile à mesurer.